In [1]:
# 📦 Install dependencies (if needed)
!pip install -q pandas scikit-learn

# 📌 Imports
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import joblib  # for saving model

# 📥 Load dataset from S3 or local
# If using S3, use: pd.read_csv("s3://your-bucket/preprocessed_dataset.csv")
df = pd.read_csv("preprocessed_dataset_0510.csv", parse_dates=["timestamp"])

# 🎯 Select features and label
features = [
    "cpu_usage", "memory_usage", "net_receive", "net_transmit",
    "fs_reads_bytes", "fs_writes_bytes", "restarts"
]
X = df[features]
y = df["label"]

# 🧼 Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 🧠 Train Isolation Forest
model = IsolationForest(
    n_estimators=100,
    contamination=0.01,  # manually chosen
    max_samples='auto',
    random_state=42
)
model.fit(X_scaled)

# 🔮 Predict
y_pred = model.predict(X_scaled)
y_pred = np.where(y_pred == -1, 1, 0)  # convert -1/1 to 1/0

# 📊 Evaluation
print("📊 Classification Report:")
print(classification_report(y, y_pred, digits=4))

print("\n📉 Confusion Matrix:")
print(confusion_matrix(y, y_pred))

# 💾 Save model & scaler for deployment
joblib.dump(model, "iforest_model.pkl")
joblib.dump(scaler, "iforest_scaler.pkl")

# 📁 Save predictions (optional)
df["predicted_label"] = y_pred
df.to_csv("iforest_predictions.csv", index=False)

print("✅ Model, scaler, and predictions saved.")

📊 Classification Report:
              precision    recall  f1-score   support

           0     0.9932    0.9910    0.9921    388059
           1     0.1077    0.1387    0.1213      3036

    accuracy                         0.9844    391095
   macro avg     0.5505    0.5648    0.5567    391095
weighted avg     0.9864    0.9844    0.9854    391095


📉 Confusion Matrix:
[[384572   3487]
 [  2615    421]]
✅ Model, scaler, and predictions saved.
